In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
df = []

Er = pd.read_excel(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\Ericsson_WLL.xlsx')
Er = Er.rename({'TCH_Congestion_Rate(Eric_Cell)':'TCH_Congestion_Rate', 'TCH_Availability(Eric_Cell)':'TCH_Availability', 
                '2G_Voice_Call_Drop_Rate(Eric_Cell)':'Voice_Call_Drop_Rate', 'TCH_Traffic(Eric_Cell)':'TCH_Traffic'},axis='columns')

Hu = pd.read_excel(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\Huawei_WLL.xlsx')
Hu = Hu.rename({'TCH_Cong(HU_Cell)':'TCH_Congestion_Rate', 'TCH_Availability(HU_Cell)':'TCH_Availability',
                '2G_Voice_Call_Drop_Rate(HU_Cell)':'Voice_Call_Drop_Rate', 'TCH_Traffic(HU_Cell)':'TCH_Traffic'},axis='columns')

No = pd.read_excel(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\Nokia_WLL.xlsx')
No = No.rename({'TCH_Cong_Rate(Nokia_Cell)':'TCH_Congestion_Rate', 'TCH_Availability(Nokia_Cell)':'TCH_Availability',
                '2G_Voice_Call_Drop_Rate(Nokia_Cell)':'Voice_Call_Drop_Rate', 'TCH_Traffic(Nokia_Cell)':'TCH_Traffic'},axis='columns')

Zt_1 = pd.read_excel(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\ZTE_WLL_1.xlsx')
Zt_1 = Zt_1.rename({'TCH_Congestion_V1':'TCH_Congestion_Rate', 'TCH_Inservicerate_AR':'TCH_Availability',
                    'Call_Drop_Ratio_V1':'Voice_Call_Drop_Rate', 'TCH_Traffic_ERL_V1':'TCH_Traffic'},axis='columns')
Zt_2 = pd.read_excel(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\ZTE_WLL_2.xlsx')
Zt_2 = Zt_2.rename({'TCH_Congestion_V2':'TCH_Congestion_Rate', 'TCH_Inservice_Rate':'TCH_Availability',
                    'Call_Drop_Ratio_V2_New':'Voice_Call_Drop_Rate', 'TCH_total_traffic_number(erl)':'TCH_Traffic'},axis='columns')


df = pd.concat([Er, Hu, No, Zt_1, Zt_2])

In [41]:
df['Vendor'] = df['ElementID'].str[-1:]
df['Index'] = df['ElementID1'].str[:2]

In [42]:
conditions = [(df['Vendor']=='W')&(df['ElementID']!='B548W'),
             (df['Vendor']=='E'),
             (df['Vendor']=='N')]

choices = ['ZTE','Ericsson','Nokia']

df['Vendor2'] = np.select(conditions, choices, default='Huawei')

df.drop(columns='Vendor',inplace=True)
df.rename(columns={'Vendor2':'Vendor'},inplace=True)

In [43]:
df['Voice_Call_Drop_Rate2'] = np.where(df['Vendor']=='ZTE',100*df['Voice_Call_Drop_Rate'],1*df['Voice_Call_Drop_Rate'])
df.drop(columns='Voice_Call_Drop_Rate',inplace=True)
df.rename(columns={'Voice_Call_Drop_Rate2':'Voice_Call_Drop_Rate'},inplace=True)

In [44]:
RecCount = df.groupby(by=['ElementID1','ElementID'],as_index=False)['TCH_Availability'].count()

In [45]:
RecCount.rename(columns={'TCH_Availability':'Count'},inplace=True)

In [46]:
df = df.merge(RecCount, how='left', on=['ElementID1','ElementID'])

In [47]:
df.sort_values(by=['ElementID1','Count'],ascending=False,inplace=True)

In [48]:
df.drop_duplicates(subset=['Date','ElementID1'],inplace=True)

In [49]:
gr10 = df[df['TCH_Traffic']>10].groupby('ElementID1',as_index=False)['TCH_Traffic'].count()
gr10.rename(columns={'TCH_Traffic':'Count>10Er'},inplace=True)

In [50]:
Avg_tr = df.groupby('ElementID1',as_index=False)['TCH_Traffic'].mean()
Avg_tr.rename(columns={'TCH_Traffic':'AvgTCH_Traffic'},inplace=True)

In [51]:
mr = pd.merge(Avg_tr, gr10, how='left', on='ElementID1')

In [52]:
df = df.merge(mr, how='left',on='ElementID1')

In [53]:
df.to_excel('D:\Saeid Lte Congested 30 March\WLL-ZTE\Sample_test3.xlsx')

In [54]:
df['Count>10Er'].value_counts()

14.0    127092
12.0      7158
13.0      6782
8.0       4552
11.0      3920
7.0       2302
10.0      1927
9.0       1478
1.0       1449
6.0       1165
2.0        994
5.0        989
3.0        811
4.0        809
Name: Count>10Er, dtype: int64

In [55]:
Avg_Drop = df.groupby('ElementID1',as_index=False)['Voice_Call_Drop_Rate'].mean()
Avg_Drop.rename(columns={'Voice_Call_Drop_Rate':'AvgVoice_Call_Drop_Rate'},inplace=True)

In [56]:
Avg_Drop['Index'] = Avg_Drop['ElementID1'].str[:2]

In [39]:
100*Avg_Drop[Avg_Drop['AvgVoice_Call_Drop_Rate']>4].groupby('Index')['Index'].count()/Avg_Drop.groupby('Index')['Index'].count()

Index
AG    0.738348
AR    3.747073
AS    0.428266
BU    4.285714
ES         NaN
FS    1.951220
GL    1.954397
GN    5.454545
HN    3.098592
HZ    3.194103
IL    2.564103
KB    1.492537
KD    0.230415
KH    2.284710
KM    5.128205
KS    0.894855
KZ    6.870229
LN    6.684492
MA    5.210918
MK         NaN
QN    0.232019
SB    6.321401
SK    3.149606
SM    2.631579
YZ    2.020202
ZN    0.194932
er         NaN
Name: Index, dtype: float64

In [57]:
Avg_Cong = df.groupby('ElementID1',as_index=False)['TCH_Congestion_Rate'].mean()
Avg_Cong.rename(columns={'TCH_Congestion_Rate':'AvgTCH_Congestion_Rate'},inplace=True)

In [58]:
Avg_Avail = df.groupby('ElementID1',as_index=False)['TCH_Availability'].mean()
Avg_Avail.rename(columns={'TCH_Availability':'AvgTCH_Availability'},inplace=True)

In [59]:
df = df.merge(Avg_Drop,how='left',on='ElementID1')
df = df.merge(Avg_Cong,how='left',on='ElementID1')
df = df.merge(Avg_Avail,how='left',on='ElementID1')

In [60]:
dropgr4 = df[df['Voice_Call_Drop_Rate']>4].groupby('ElementID1',as_index=False)['Voice_Call_Drop_Rate'].count()
dropgr4.rename(columns={'Voice_Call_Drop_Rate':'drop>4'},inplace=True)

In [61]:
conggr2 = df[df['TCH_Congestion_Rate']>2].groupby('ElementID1',as_index=False)['TCH_Congestion_Rate'].count()
conggr2.rename(columns={'TCH_Congestion_Rate':'cong>2'},inplace=True)

In [62]:
availles90 = df[df['TCH_Availability']<90].groupby('ElementID1',as_index=False)['TCH_Availability'].count()
availles90.rename(columns={'TCH_Availability':'avail<90'},inplace=True)

In [63]:
df = df.merge(dropgr4,how='left',on='ElementID1')

In [64]:
df = df.merge(conggr2,how='left',on='ElementID1')
df = df.merge(availles90,how='left',on='ElementID1')

In [65]:
df_drop = []
df_cong = []
df_avail = []

In [66]:
df_drop = df[df['drop>4']>6]
df_cong = df[df['cong>2']>6]
df_avail = df[df['avail<90']>6]

In [67]:
df_drop.drop(columns=['Date','TCH_Congestion_Rate','TCH_Availability','TCH_Traffic','Voice_Call_Drop_Rate',
                     'AvgTCH_Congestion_Rate','AvgTCH_Availability','cong>2','avail<90'],inplace=True)

df_drop.drop_duplicates(inplace=True)

c:\users\milad\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
c:\users\milad\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [68]:
df_cong.drop(columns=['Date','TCH_Congestion_Rate','TCH_Availability','TCH_Traffic','Voice_Call_Drop_Rate',
                     'AvgTCH_Availability','AvgVoice_Call_Drop_Rate','avail<90','drop>4'],inplace=True)

df_cong.drop_duplicates(inplace=True)

c:\users\milad\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [69]:
df_avail.drop(columns=['Date','TCH_Congestion_Rate','TCH_Availability','TCH_Traffic','Voice_Call_Drop_Rate',
                     'AvgTCH_Congestion_Rate','AvgVoice_Call_Drop_Rate','cong>2','drop>4'],inplace=True)

df_avail.drop_duplicates(inplace=True)

c:\users\milad\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [76]:
writer = pd.ExcelWriter(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\result2.xlsx', engine='xlsxwriter')
df_drop.to_excel(writer,sheet_name='Drop')
df_cong.to_excel(writer,sheet_name='Congestion')
df_avail.to_excel(writer,sheet_name='Availability')
writer.save()

In [74]:
100*df_drop.groupby('Index')['Index'].count()/Avg_Drop.groupby('Index')['Index'].count()

KeyError: 'Index'

In [73]:
Avg_Drop.groupby('Index')['Index'].count()

Index
AG    2167
AR     427
AS    1401
BU      70
ES     205
FS     615
GL     307
GN     165
HN     355
HZ     407
IL      39
KB     201
KD     434
KH     569
KM     351
KS     447
KZ     393
LN     374
MA     403
MK     207
QN     431
SB    1313
SK     254
SM      76
YZ      99
ZN     513
er       3
Name: Index, dtype: int64

In [72]:
Avg_Cong['Index'] = Avg_Cong['ElementID1'].str[:2]
Avg_Avail['Index'] = Avg_Avail['ElementID1'].str[:2]

In [75]:
Avg_Cong.groupby('Index')['Index'].count()

Index
AG    2167
AR     427
AS    1401
BU      70
ES     205
FS     615
GL     307
GN     165
HN     355
HZ     407
IL      39
KB     201
KD     434
KH     569
KM     351
KS     447
KZ     393
LN     374
MA     403
MK     207
QN     431
SB    1313
SK     254
SM      76
YZ      99
ZN     513
er       3
Name: Index, dtype: int64

In [125]:
dr_dash = 100*df_drop.groupby('Index')['Index'].count()/Avg_Drop.groupby('Index')['Index'].count()
cong_dash = 100*df_cong.groupby('Index')['Index'].count()/Avg_Cong.groupby('Index')['Index'].count()
avail_dash = 100*df_avail.groupby('Index')['Index'].count()/Avg_Avail.groupby('Index')['Index'].count()

In [128]:
writer = pd.ExcelWriter(r'D:\Saeid Lte Congested 30 March\WLL-ZTE\dash.xlsx', engine='xlsxwriter')
dr_dash.to_excel(writer,sheet_name='Drop')
cong_dash.to_excel(writer,sheet_name='Congestion')
avail_dash.to_excel(writer,sheet_name='Availability')
writer.save()